In [ ]:
%pip install requests beautifulsoup4 pandas lxml IMDbPY

In [42]:

#imports
import requests
from bs4 import BeautifulSoup
from imdb import IMDb
import pandas as pd
import time

#Scrapping: getting RT editoral URL content and feeding it to BS
url = 'https://editorial.rottentomatoes.com/guide/best-movies-of-all-time/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')

#empty list to store movie info
movies = []

#Targeting all table of RT page (this is where the movie information is)
tables = soup.find_all('table', class_='aligncenter')
#print(tables)

#iterate through all tables
for table in tables:
  rows = table.find_all('tr')
  for row in rows:
    try:
      title = row.find('a', class_='title').text.strip()
      year = row.find('span', class_='year').text.strip().strip('()')
      rt_rating = row.find('span', class_='score').text.strip()
      
      movies.append({'title':title,'year':year,'RT rating':rt_rating})
      
    except Exception:
      continue


#Turn the movie dict stored in the movies list data frame (table)
df = pd.DataFrame(movies)
print(df.head())










               title  year RT rating
0      The Godfather  1972       97%
1         Casablanca  1942       99%
2  L.A. Confidential  1997       99%
3      Seven Samurai  1954      100%
4           Parasite  2019       99%


In [ ]:
import time

#Instance of IMDb
im = IMDb()

#new columns to the table
df['genres'] = ''
df['plot'] = ''


#Enriching df table with more information
for idx, row in df.iterrows():
  title = row['title']
  year = row['year']
  
  try:
    results = im.search_movie(title)
    if results:
      movie = results[0]
      im.update(movie)
      
      df.at[idx, 'genres'] = ','.join(movie.get('genres', []))
      df.at[idx, 'plot'] = movie.get('plot', [''])[0]
      
      print(f'{title} enriched')
    else:
      print(f'No match for {title}')
      
    time.sleep(0.5)
    
  except Exception as e:
    print(f'Error for {title}: {e}')
    
    continue
      
      
      
      
  

In [ ]:

print(df.head())

df.to_csv('movie_info.csv', index=False, encoding="utf-8")


In [ ]:
omdb_api_key = "b2050bec"
df = pd.read_csv('main_movie_info.csv')

#function to use OMDB
def get_omdb_info(title, year):
  url = f"http://www.omdbapi.com/?t={title}&y={year}&apikey={omdb_api_key}"
  response = requests.get(url)
  if response.status_code == 200:
        data = response.json()
        if data.get("Response") == "True":
            return {
                "genre": data.get("Genre", ""),
                "runtime": data.get("Runtime", ""),
                "imdb_rating": data.get("imdbRating", "")
            }
  return None

# Ensure the columns exist
for col in ["genre", "runtime", "imdb_rating"]:
    if col not in df.columns:
        df[col] = ""

for idx, row in df.iterrows():
    result = get_omdb_info(row["title"], row["year"])
    if result:
        df.at[idx, "genre"] = result["genre"]
        df.at[idx, "runtime"] = result["runtime"]
        df.at[idx, "imdb_rating"] = result["imdb_rating"]
        print(f"{row['title']} updated.")
    else:
        print(f"{row['title']} not found.")
    
    time.sleep(0.3)  # OMDb free tier allows 1 request/second max


In [67]:
df.to_csv('main_movie_info.csv', index=False)